In [1]:
# !pip install rank-bm25

In [2]:
# 1. 초기 설정 및 Chroma DB 연결
import torch
import chromadb
from sentence_transformers import SentenceTransformer

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# Chroma DB 연결
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_collection(name="patent_claims")

c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 3. BM25 모델 생성
from rank_bm25 import BM25Okapi

documents = collection.get()["documents"]

# 토큰화(청구항)
tokenized_docs = [doc.split() for doc in documents]

# BM25 생성
bm25 = BM25Okapi(tokenized_docs)                

In [4]:
# 4. doc_id → BM25 인덱스 매핑
# 문서 ID와 BM25 순서를 맞춰야 함
doc_id_to_idx = {}
raw_ids = collection.get()["ids"]

for idx, doc_ids in enumerate(raw_ids):
    for doc_id in doc_ids:
        doc_id_to_idx[doc_id] = idx

In [10]:
# 5. 하이브리드 서치 함수 정의
def hybrid_search(query, top_k=30):

    if isinstance(query, list):
        results_list = []
        for q in query:
            results_list.append(hybrid_search(q, top_k))
        return results_list

    # 1) 벡터 검색
    query_vector = model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )

    vector_docs = results["documents"][0]
    vector_ids = results["ids"][0]  
    vector_metas = results["metadatas"][0]
    vector_scores = [1 - d for d in results["distances"][0]] 

    # 2) BM25 검색
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query)  # 각 문서별 점수

    # 3) 점수 합산
    hybrid_list = []
    for doc_text, meta, doc_id, v_score in zip(vector_docs, vector_metas, vector_ids, vector_scores):

        # BM25 점수 찾기 
        if doc_id in doc_id_to_idx:
            bm25_idx = doc_id_to_idx[doc_id]
            b_score = bm25_scores[bm25_idx]
        else:
            b_score = 0  # 샘플에 없는 경우

        final_score = 0.7 * v_score + 0.3 * b_score

        hybrid_list.append({
            "text": doc_text,
            "patent_id": meta.get("patent_id"),   
            "claim_no": meta.get("claim_no"),
            "score": final_score
        })


    # 4) 점수로 최종 정렬
    hybrid_list.sort(key=lambda x: x["score"], reverse=True)

    return hybrid_list


In [6]:
results = hybrid_search("영상 객체 인식")

for r in results:
    print(f"출원번호: {r['patent_id']}, 점수: {r['score']:.4f}")
    print(f"내용: {r['text'][:120]}...\n")

출원번호: 1020230137106, 점수: 0.4659
내용: 객체를 인식하는 객체 인식 장치에 의해 수행되는 객체 인식 방법에 있어서,RGB 형식의 입력 영상을 HSV 형식의 영상으로 변환하는 동작;상기 HSV 형식의 영상에서 명도 값을 추출하는 동작;기준 명도 값에 기초하여...

출원번호: 1020220016940, 점수: 0.4406
내용: 상기 비정형 객체를 인식하는 단계는,상기 동영상에서 상기 이동 객체 좌표에 따른 객체 바운딩박스를 제외시키고, 영상전환점을 다시 추출하는 단계; 및다시 추출된 영상전환점 이후의 동영상에 대하여 커스텀 학습 모델을 적...

출원번호: 1020230037336, 점수: 0.4405
내용: 상기 객체 인식단계는,상기 최대 윤곽선 영역을 객체로 인식하는 것을 특징으로 하는 마커리스 기반 보행영상의 객체 인식 방법....

출원번호: 1020210044655, 점수: 0.4362
내용: 객체 인식 서버에 의해 수행되는 영상 데이터에 포함된 복수의 객체를 인식하는 방법에 있어서, 기구축된 머신러닝 모델을 통해 영상 데이터로부터 기설정된 객체를 인식하는 객체 인식 프로세스를 수행하는 단계;상기 인식된 ...

출원번호: 1020220016940, 점수: 0.4348
내용: 상기 영상전환점을 추출하는 단계는,상기 동영상에서 콘텐츠 인식 감지로 두 개의 후속 프레임간의 차이가 임계값을 초과하는 영역을 찾는 방식으로 수행되는 비정형객체 인식 방법....

출원번호: 1020190105256, 점수: 0.4346
내용: 상기 동영상 출현 객체를 인식하는 단계는,지도 학습을 통해 훈련된 인공지능 모델을 이용하여 프레임에 출현하는 객체를 인식하는 단계를 포함하고,상기 인공지능 모델은,빅데이터(big data)를 이용한 초기 학습에 추가...

출원번호: 1020220123363, 점수: 0.4342
내용: 차량의 주변을 촬영한 동영상에서 경광등을 구비한 긴급 차량의 객체 인식을 수행하는 단계와;상기 동영상의 단일의 프레임을 이용하여 컴

---

- 출원번호를 뽑아서 정답 출원번호와 교집합 몇 개인지

In [7]:
# 같은 ipcNumber을 가진 출원번호
ipc_g05d143 = {"1020240014135",
"1020230114322",
"1020250071099",
"1020257016601",
"1020240039664",
"1020240037602",
"1020240024079",
"1020230147397",
"1020257029654",
"1020240062392",
"1020257011052",
"1020220110142",
"1020220132771",
"1020230173300",
"1020240085547",
"1020250040520",
"1020240013189",
"1020247037212",
"1020257009077",
"1020247018578",
"1020247018568",
"1020247018428",
"1020240079243",
"1020237031214",
"1020240026208",
"1020230177082",
"1020220120962",
"1020240003758",
"1020230134552",
"1020230127447",
"1020230127436"}

In [17]:
def eval(query, ipc_list, TOP_K=10):
    final_response = hybrid_search(query)

    # 비교
    # flatten
    flat_response = [item for sublist in final_response for item in sublist]

    response_patent_id = [f['patent_id'] for f in flat_response]

    print(ipc_list & set(response_patent_id))
    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)

In [20]:
query = [
    "주행 로봇의 위치·방향을 판단하고 제어하는 알고리즘",
    "지도 기반 위치 결정 및 경로 추종 제어 로직"
]

TOP_K = 100
eval(query, ipc_g05d143, TOP_K)

{'1020240003758'}
Precision: 0.01
